# Cómo pivotar y despivotar un DataFrame de Spark

La función de Spark, ```pivot()```, se utiliza para pivotar/rotar los datos de una columna del DataFrame/Dataset en múltiples columnas (transformar fila en columna) y ```unpivot()``` se utiliza para transformarlos de nuevo (transformar columnas en filas).

```Pivot()``` es una agregación en la que uno de los valores de las columnas de agrupación se transpone en columnas individuales con datos distintos.

Vamos a crear un DataFrame para trabajar con él.

In [1]:
val data = Seq(("Banana",1000,"USA"), ("Carrots",1500,"USA"), ("Beans",1600,"USA"),
      ("Orange",2000,"USA"),("Orange",2000,"USA"),("Banana",400,"China"),
      ("Carrots",1200,"China"),("Beans",1500,"China"),("Orange",4000,"China"),
      ("Banana",2000,"Canada"),("Carrots",2000,"Canada"),("Beans",2000,"Mexico"))

import spark.sqlContext.implicits._

val df = data.toDF("Product","Amount","Country")

df.show()

Intitializing Scala interpreter ...

Spark Web UI available at http://ALC-1NJW5D3.usersad.everis.int:4041
SparkContext available as 'sc' (version = 3.3.0, master = local[*], app id = local-1656923599632)
SparkSession available as 'spark'


22/07/04 10:33:34 WARN ProcfsMetricsGetter: Exception when trying to compute pagesize, as a result reporting of ProcessTree metrics is stopped
+-------+------+-------+
|Product|Amount|Country|
+-------+------+-------+
| Banana|  1000|    USA|
|Carrots|  1500|    USA|
|  Beans|  1600|    USA|
| Orange|  2000|    USA|
| Orange|  2000|    USA|
| Banana|   400|  China|
|Carrots|  1200|  China|
|  Beans|  1500|  China|
| Orange|  4000|  China|
| Banana|  2000| Canada|
|Carrots|  2000| Canada|
|  Beans|  2000| Mexico|
+-------+------+-------+



data: Seq[(String, Int, String)] = List((Banana,1000,USA), (Carrots,1500,USA), (Beans,1600,USA), (Orange,2000,USA), (Orange,2000,USA), (Banana,400,China), (Carrots,1200,China), (Beans,1500,China), (Orange,4000,China), (Banana,2000,Canada), (Carrots,2000,Canada), (Beans,2000,Mexico))
import spark.sqlContext.implicits._
df: org.apache.spark.sql.DataFrame = [Product: string, Amount: int ... 1 more field]


## Pivotar Spark DataFrame
Spark SQL proporciona la función ```pivot()``` para girar los datos de una columna en múltiples columnas (transponer fila a columna). Se trata de una agregación en la que uno de los valores de las columnas de agrupación se transpone en columnas individuales con datos distintos. A partir del DataFrame anterior, para obtener la cantidad total exportada a cada país de cada producto se hará la agrupación por 'Product', el pivote por 'Country', y la suma de 'Amount'.

Esto transpondrá los países de las filas del DataFrame a las columnas. Cuando los datos no están presentes, se representan como nulos por defecto.

In [2]:
val pivotDF = df.groupBy("Product").pivot("Country").sum("Amount")
pivotDF.show()

+-------+------+-----+------+----+
|Product|Canada|China|Mexico| USA|
+-------+------+-----+------+----+
| Orange|  null| 4000|  null|4000|
|  Beans|  null| 1500|  2000|1600|
| Banana|  2000|  400|  null|1000|
|Carrots|  2000| 1200|  null|1500|
+-------+------+-----+------+----+



pivotDF: org.apache.spark.sql.DataFrame = [Product: string, Canada: bigint ... 3 more fields]


## Mejora del rendimiento de Pivot en Spark 2.0
El rendimiento de Spark 2.0 en adelante ha sido mejorado en Pivot, sin embargo, si estás usando una versión inferior; ten en cuenta que el pivote es una operación muy costosa por lo tanto, se recomienda proporcionar los datos de la columna (si se conocen) como un argumento a la función como se muestra a continuación.

In [3]:
val countries = Seq("USA","China","Canada","Mexico")
val pivotDF = df.groupBy("Product").pivot("Country", countries).sum("Amount")
pivotDF.show()

+-------+----+-----+------+------+
|Product| USA|China|Canada|Mexico|
+-------+----+-----+------+------+
| Orange|4000| 4000|  null|  null|
|  Beans|1600| 1500|  null|  2000|
| Banana|1000|  400|  2000|  null|
|Carrots|1500| 1200|  2000|  null|
+-------+----+-----+------+------+



countries: Seq[String] = List(USA, China, Canada, Mexico)
pivotDF: org.apache.spark.sql.DataFrame = [Product: string, USA: bigint ... 3 more fields]


Otro enfoque es hacer una agregación en dos fases. Spark 2.0 utiliza esta implementación para mejorar el rendimiento Spark-13749

In [4]:
val pivotDF = df.groupBy("Product","Country")
      .sum("Amount")
      .groupBy("Product")
      .pivot("Country")
      .sum("sum(Amount)")
pivotDF.show()

+-------+------+-----+------+----+
|Product|Canada|China|Mexico| USA|
+-------+------+-----+------+----+
| Orange|  null| 4000|  null|4000|
|  Beans|  null| 1500|  2000|1600|
| Banana|  2000|  400|  null|1000|
|Carrots|  2000| 1200|  null|1500|
+-------+------+-----+------+----+



pivotDF: org.apache.spark.sql.DataFrame = [Product: string, Canada: bigint ... 3 more fields]


Los dos ejemplos anteriores devuelven el mismo resultado pero con mejor rendimiento.
## Unpivot Spark DataFrame
Unpivot es una operación inversa, que podemos lograr girando los valores de las columnas en los valores de las filas. Spark SQL no tiene la función unpivot, por lo que utilizará la función ```stack()```. El código siguiente convierte los países de las columnas en filas.

In [5]:
//Unpivot
val unPivotDF = pivotDF.select($"Product",
expr("stack(3, 'Canada', Canada, 'China', China, 'Mexico', Mexico) as (Country,Total)"))
.where("Total is not null")
unPivotDF.show()

+-------+-------+-----+
|Product|Country|Total|
+-------+-------+-----+
| Orange|  China| 4000|
|  Beans|  China| 1500|
|  Beans| Mexico| 2000|
| Banana| Canada| 2000|
| Banana|  China|  400|
|Carrots| Canada| 2000|
|Carrots|  China| 1200|
+-------+-------+-----+



unPivotDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Product: string, Country: string ... 1 more field]


## Transposición o pivote sin agregación
¿Podemos hacer la transposición o el pivote de Spark DataFrame sin agregación?

Por supuesto que se puede, pero desafortunadamente, no se puede lograr usando la función Pivot. Sin embargo, pivotar o transponer la estructura del DataFrame sin agregación de filas a columnas y de columnas a filas se puede hacer fácilmente usando Spark y Scala hack.

In [6]:
val df = Seq(
  ("col1", "val1"),
  ("col2", "val2"),
  ("col3", "val3"),
  ("col4", "val4"),
  ("col5", "val5")
).toDF("COLUMN_NAME", "VALUE")

df.show()

+-----------+-----+
|COLUMN_NAME|VALUE|
+-----------+-----+
|       col1| val1|
|       col2| val2|
|       col3| val3|
|       col4| val4|
|       col5| val5|
+-----------+-----+



df: org.apache.spark.sql.DataFrame = [COLUMN_NAME: string, VALUE: string]


In [7]:
df.groupBy()
  .pivot("COLUMN_NAME").agg(first("VALUE"))
  .show()

+----+----+----+----+----+
|col1|col2|col3|col4|col5|
+----+----+----+----+----+
|val1|val2|val3|val4|val5|
+----+----+----+----+----+



Si el DataFrame es realmente tan pequeño como en el ejemplo, podemos recogerlo como Map:

In [8]:
val map = df.as[(String,String)].collect().toMap

print(map)

Map(col3 -> val3, col2 -> val2, col5 -> val5, col1 -> val1, col4 -> val4)

map: scala.collection.immutable.Map[String,String] = Map(col3 -> val3, col2 -> val2, col5 -> val5, col1 -> val1, col4 -> val4)


## Conclusión
Hemos visto cómo Pivotar DataFrame (transponer fila a columna) con el ejemplo de scala y Unpivot de nuevo usando las funciones SQL de Spark. Y también hemos visto cómo los cambios de Spark 2.0 mejoran el rendimiento al hacer la agregación en dos fases.